In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D

import xspec as x

In [2]:
x.Xset.chatter = 0
#x.Xset.logChatter = 25
logFile = x.Xset.openLog("newLogFile.txt")

In [10]:
x.AllData.clear()
x.AllData.removeDummyrsp()
x.AllData.dummyrsp(lowE=0.1, highE=10.0, nBins=100)
x.Xset.addModelString("APEC_TRACE_ABUND", "0")

x.Plot.device = "/xs"

In [107]:
temperatures = (1,10) #np.logspace(np.log10(0.01), np.log10(10), 101)

Tt = 1.0
Ab = 1.0
Norm = 1
z = 0
n_H = 0.00

headers = ['Flux', 'Abund', 'T', 'z', 'n_H', 'Chnls', '$E_{min}$', '$E_{max}$', '$E_{sum}$', 'cs', 'ecs' ]
df = pd.DataFrame([], columns = headers)

for Temp in temperatures:
    mod = x.Model('cflux*phabs*(apec+const*apec)')
    mod.setPars(0.4, 2.0, -12., n_H, Temp, Ab, z, Norm, 1., Temp, 0.0, z, Norm)
    mod(9).values =  "-1, 0.0001, -1, -1, 1, 1"
    mod(10).link = "5"
    mod(12).link = "7"
    mod(13).link = "8"

    x.AllModels.show()
    x.Plot.show()
    #x.AllModels.setEnergies("0.1 10.0 10 log")
    #x.AllModels.setEnergies("reset")
    #x.Plot("model")
    
    fs = x.FakeitSettings(response = 'telescopes/chandra/djs50.ugc3957_v05.rmf', 
                           arf = 'telescopes/chandra/djs50.ugc3957_v05.arf', 
                    background = '', 
                      exposure = 40000, 
                    correction = '', 
                  backExposure = '', 
                      fileName = 'fakeit.pha')
    x.AllData.fakeit(nSpectra = 1, 
                     settings = fs, 
                   applyStats = True,
                   filePrefix = "",
                      noWrite = True)

    x.AllData.ignore("**-0.2 8.0-**")             # IMPORTANT !
    x.AllData.show()
    x.AllModels.setEnergies("reset")
    
    x.Plot("ldata")
    x.Plot.xAxis = "keV"
    xVals = x.Plot.x()
    yVals = x.Plot.y()
    print('np.dot', np.dot(xVals, yVals))
    
    
    x.AllModels.calcFlux('0.2 8.0')
    fluxx = x.AllData(1).flux[0]
    print(fluxx)
    
    channels = len(x.AllData(1).noticed)
    print(channels)
    
    ens = x.AllData(1).energies
    e_min = min(ens)[0]
    e_max = max(ens)[1]
    print(e_min, e_max)
    
    s0 = 0
    s1 = 0
    sm = np.zeros(len(ens))
    for i in ens:
        s0 += i[0]
        s1 += i[1]
        sm[ens.index(i)] = (i[0]+i[1])/2
        #print(s0, s1, sm)
    e_sum = (s0+s1)/2
    print(e_sum, sum(sm))
    
    cs = x.AllData(1).rate
    print(cs[2])
    
    r = x.AllData(1).values
    print(np.dot(r, sm)*40000)
    
    #df2 = pd.DataFrame([[fluxx, Ab, Temp, z, n_H, channels, e_min, e_max
        
        
     #                   ]], columns = cols, index=None)
            
   # df = pd.concat([df, df2])
    
df
df.to_csv('stats.dat', sep=' ', index=False)


User entered plot commands:
np.dot 13.448061832568744
1.1114193364590479e-12
533
0.20440000295639038 7.986200332641602
2182.794950827956 2182.794950827956
0.18012499999999962
7853.668688714503

User entered plot commands:
np.dot 46.524300342083265
4.519377031844587e-12
533
0.20440000295639038 7.986200332641602
2182.794950827956 2182.794950827956
0.27514999999999895
27170.191823422905


2182.794950827956


In [ ]:

        
        # fitting
        x.AllModels.clear()
        mod2fit = x.Model("phabs*(apec+const*apec)")
        mod2fit.setPars(0.0, 1.0, abund, 0., nrm, 1, 1., 0.0, 0.0, nrm)
        mod2fit(1).frozen = True   # n_H
        #mod2fit(2).values = f"{(T_minnn+T_maxxx)/2}, 0.001, {T_minnn}, {T_minnn}, {T_maxxx}, {T_maxxx}" # temperature
        mod2fit(3).frozen = False  # abundance
        #mod2fit(4).frozen = False  # redshift
        mod2fit(5).frozen = True   # norm
        mod2fit(5).values = f"{nrm}, -1, 0.0, 0.0, 1.1, 1.1"
        mod2fit(6).frozen = True   # const = -1
        mod2fit(6).values = "-1, -1, -1, -1, 1, 1"
        mod2fit(7).link = "2"      # temperature
        mod2fit(8).frozen = True   # zero abund for continuum
        mod2fit(9).link = "4"      # redshift
        mod2fit(10).link = "5"     # norm
        
        #x.AllData.ignore("bad")
        x.Fit.renorm('auto')
        x.Fit.nIterations = 100
        x.Fit.query = 'yes'
        x.Fit.weight = 'standard'
        x.Fit.statMethod = 'cstat'
        #x.Fit.delta = 0.001
        x.Fit.perform()
        #x.Fit.goodness(100, sim=False)
        x.Fit.show()
               
        # steppar
        if stpar:
            N_steps = 30
            perform_steppar(mod2fit, 2, 0.2, 3, 0.2, N_steps)
            
        if nrm == 0.011:
            x.Fit.error('2')
            dleft, dright, _ = mod2fit(2).error
            #print(mod2fit(2).error)
            dt_lefts.append(dleft)
            dt_rights.append(dright)

        # return some parameters
        best_kT = mod2fit(2).values[0]
        abund_from_fit = mod2fit(3).values[0]
        norm = mod2fit(5).values[0]
        tspec_list.append(best_kT)
        #print(best_kT)
        
        # calculating flux
        x.AllModels.calcFlux('0.7 10.0')
        fluxx = x.AllData(1).flux[0]
        flux_list.append(fluxx) # in units of ergs/cm2/s 
        #or use [4] in units of photons / s / cm^2
    
        if plot:
            plt.suptitle(f'$T_{{min}}={T_minnn} \ keV, \ T_{{max}}={T_maxxx} \ keV, \ f_{{min}}={f_minnn:.2f}, \ f_{{max}}={f_maxxx:.2f}$ \n $t_{{exp}}={texp} \ s, \ Z_{{model}} ={abund}, \ Z_{{from \ fit}} = {abund_from_fit:.2f}, \ norm = {norm:.4f}$ \n $T_{{spec}}={best_kT:.4f} \ keV, \ $Flux$ = {fluxx*10**12:.3f}\cdot 10^{{-12}} \ ergs/cm^2/s$ \n ', fontsize = 20)
            plt.subplot(3,2,3)
            if stpar:
                plot_contours_from_steppar(N_steps, 2, 3, mod2fit, zoomin=False)
            #else:
                #draw_best_model(nrm, linesandcont=True)
                #draw_goodness()
            plt.subplot(3,2,2)
            draw_data_and_best_model(nrm, linesandcont=True)
            plt.subplot(3,2,1)
            draw_best_model(nrm, linesandcont=False)
            plt.show()

        #x.Plot.commands=()
        x.AllData.clear()
        x.AllModels.clear()

    return tspec_list, flux_list, dt_lefts, dt_rights